In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nature/Nature.csv
/kaggle/input/nature/Fantasy.csv
/kaggle/input/fantasy-model/Fantasy_Model.h5
/kaggle/input/nature-model/Nature_Model.h5
/kaggle/input/othertopics-model/otherTopics_Model.h5
/kaggle/input/othertopics/OtherTopics.csv
/kaggle/input/emotions-model/Emotions_Model.h5
/kaggle/input/seasons-nlp/Seasons_Model.h5
/kaggle/input/virtues-model/Virtues_Model.h5
/kaggle/input/seasons/Seasons.csv
/kaggle/input/virtues/Virtues.csv
/kaggle/input/emotions/Emotions.csv


**Importing Libraries**

In [2]:
import numpy as np
import pandas as pd
import nltk
from nltk import pos_tag
from nltk import RegexpParser
from nltk.corpus import wordnet as wn
import matplotlib.pyplot as plt
import tensorflow.keras.utils as ku
from wordcloud import WordCloud
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from keras.models import load_model

**Generating the Poem Lines**

In [3]:
def GenerateLines(ModelPath, DataPath, SplitVal, Seed, max_sequence_len):
    seed_text = Seed
    next_words = 6
    ouptut_text = ""
    
    data = pd.read_csv(DataPath)
    
    text_data = ""
    for index, row in data.iterrows():
        text_data = text_data + row['text']
    
    corpus = text_data.lower().split("\n")
    corpus = corpus[::SplitVal]
    
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(corpus)
    
    model = load_model(ModelPath)
    
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)
        output_word = ""

        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break

        seed_text += " " + output_word
    
    corpus = []
    text_data = ""
    
    return(seed_text)

**Including Wordnet**

In [4]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [5]:
wn.synsets("happy")[0].also_sees() 

[Synset('cheerful.a.01'),
 Synset('contented.a.01'),
 Synset('elated.a.01'),
 Synset('euphoric.a.01'),
 Synset('felicitous.a.01'),
 Synset('glad.a.01'),
 Synset('joyful.a.01'),
 Synset('joyous.a.01')]

**Obtaining the Most SImilar Category**

In [5]:
CATEGORIES = {
    'emotions': [
        'sorrow',
        'greed',
        'fear',
        'laughter',
        'love',
        'hate',
        'happy',
        'sympathy',
        'depression',
        'hope',
        'anger'
    ], 
    'virtues': [
        'respect',
        'truth',
        'justice',
        'peace',
        'freedom',
        'courage',
        'innocence',
        'beauty',
        'lonely', 
        'trust'
    ],
    'nature': [
        'butterfly',
        'snake',
        'rose',
        'moon',
        'sun',
        'sky',
        'star',
        'rain',
        'rainbow',
        'green',
        'nature',
        'animal',
        'fire', 
        'water'
    ], 
    'fantasy': [
        'angel',
        'hero', 
        'heaven', 
        'god',
        'dream',
        'believe', 
        'faith', 
        'time',
        'dark',
        'destiny'
    ],
    'seasons': [
      'january', 
      'june', 
      'june',
      'christmas',
       'spring',
       'winter',
       'summer',
      'weather',
    ], 
    'other': [
        'sometimes',  
        'future',  
        'mirror',    
        'remember', 
        'today', 
        'power', 
        'together',  
        'red',  
        'success', 
        'crazy', 
        'memory', 
        'money',   
        'food', 
        'car', 
        'night', 
    ]
}

def getCategory(NounWord):
    maximum = 0
    Category = ""
    sum1 = 0
    if(len(wn.synsets(NounWord, 'n')) > 0):
        cat = wn.synsets(NounWord, 'n')[0]
        for j in ['emotions', 'virtues', 'nature', 'fantasy', 'seasons', 'other']:
            sum1 = 0
            for i in CATEGORIES[j]:
                if(len(wn.synsets(i, 'n')) > 0):
                    x = cat.wup_similarity(wn.synsets(i, 'n')[0])
                    #print(i, x)
                    sum1 = sum1 + x
            if(sum1 > maximum):
                maximum = sum1
                Category = j
    return(Category, sum1)
#argmax Ej Ei similarity(keyword, j[i])

**Chunking and Obtaining Key Words**

In [6]:
def ChunkingSentence(Sentence):
    text = Sentence.split()
    #POS Tags
    POS_tag = pos_tag(text)
    print(POS_tag)
    NounTags = []
    for i in POS_tag:
        if i[1].startswith("NN"):
            temp = i[0]
            NounTags.append(temp)
        elif i[1].startswith("VB"):
            temp = i[0]
            NounTags.append(temp)
    return(NounTags)

**Input Sentence for Poem**

In [24]:
Query = input("Enter the kind of poem you want: ")
KeyWords = ChunkingSentence(Query)

Enter the kind of poem you want:  about a boy who likes girls


[('about', 'IN'), ('a', 'DT'), ('boy', 'NN'), ('who', 'WP'), ('likes', 'VBZ'), ('girls', 'NNS')]


In [25]:
D = dict()
for word in KeyWords:
    Cat, Sum1 = getCategory(word)
    if(Cat in D):
        D[Cat] = D[Cat] + Sum1
    else:
        D[Cat] = Sum1
print(D)

Keymax = max(zip(D.values(), D.keys()))[1]
print(Keymax)

{'nature': 6.0429529534792685, 'fantasy': 2.899030664897538}
nature


**Obtaining the Seed Words**

In [10]:
for i in KeyWords:
    #print(wn.synsets("girl")[0].also_sees()) 
    '''for syn in wn.synsets(i):
        print(syn.hypernyms())'''
    print(wn.synsets(i))

[Synset('friend.n.01'), Synset('ally.n.02'), Synset('acquaintance.n.03'), Synset('supporter.n.01'), Synset('friend.n.05')]
[Synset('internet.n.01')]


In [26]:
Seeds = []
for i in KeyWords:
    #print(wn.synsets("girl")[0].also_sees()) 
    for syn in wn.synsets(i):
        for lemma in syn.lemmas():
            Seeds.append(lemma.name())

CleanSeeds = []
for i in Seeds:
    x = i.replace("_", " ")
    CleanSeeds.append(x)
Seeds = []
print(set(CleanSeeds))

{'fille', 'young lady', 'wish', 'the like', 'male child', 'son', 'lady friend', 'the likes of', 'young woman', 'female child', 'like', 'boy', 'care', 'girlfriend', 'ilk', 'missy', 'daughter', 'miss', 'girl', 'little girl'}


In [12]:
import gensim.downloader
print(list(gensim.downloader.info()['models'].keys()))

# Download the "glove-twitter-25" embeddings
glove_vectors = gensim.downloader.load('glove-twitter-25')

for i in KeyWords:
    print(glove_vectors.most_similar(i))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']
[==================================================] 100.0% 104.8/104.8MB downloaded


KeyboardInterrupt: 

**Actually choosing 4 Seed Words**

In [30]:
# importing random module
import random

# initializing the value of n
n = 4
 
# printing n elements from list
RandSeeds = random.sample(CleanSeeds, n)
print(RandSeeds)

['boy', 'female child', 'like', 'wish']


**Calling the Generate Line function with seed words**

In [15]:
def MakePoem(ModelPath, DataPath, SplitVal,EmbedLen):
    for i in RandSeeds:
        Output = ""
        Output = GenerateLines(ModelPath, DataPath, SplitVal, i, EmbedLen)
        print(Output)

In [31]:
ModelPath = "/kaggle/input/emotions-model/Emotions_Model.h5"
DataPath = "/kaggle/input/emotions/Emotions.csv"
if(Keymax == 'emotions'):
    MakePoem(ModelPath, DataPath, 4, 235)
elif(Keymax == 'fantasy'):
    ModelPath = "/kaggle/input/fantasy-model/Fantasy_Model.h5"
    DataPath = "/kaggle/input/nature/Fantasy.csv"
    MakePoem(ModelPath, DataPath, 5, 445)
elif(Keymax == 'nature'):
    ModelPath = "/kaggle/input/nature-model/Nature_Model.h5"
    DataPath = "/kaggle/input/nature/Nature.csv"
    MakePoem(ModelPath, DataPath, 7, 263)
elif(Keymax == 'seasons'):
    ModelPath = "/kaggle/input/seasons-nlp/Seasons_Model.h5"
    DataPath = "/kaggle/input/seasons/Seasons.csv"
    MakePoem(ModelPath, DataPath, 5, 255)
elif(Keymax == 'virtues'):
    ModelPath = "/kaggle/input/virtues-model/Virtues_Model.h5"
    DataPath = "/kaggle/input/virtues/Virtues.csv"
    MakePoem(ModelPath, DataPath, 3, 117)
elif(Keymax == 'other'):
    ModelPath = "/kaggle/input/othertopics-model/otherTopics_Model.h5"
    DataPath = "/kaggle/input/othertopics/OtherTopics.csv"
    MakePoem(ModelPath, DataPath, 9, 2113)

boy tributaries flimsily hour bound us hugs
female child below the sea rief charmer come
like the moon is on your fingers
wish the river is dotted in cries
